<a href="https://colab.research.google.com/github/athipan1/Database_painaidee/blob/main/Database_Painaidee_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏛️ Database Painaidee - Thai Attractions API

## ระบบจัดการข้อมูลสถานที่ท่องเที่ยวไทย / Thai Attractions Database Management System

This notebook allows you to run the Database Painaidee Flask API on Google Colab with ngrok tunneling for external access.

**Features:**
- 🏛️ Thai attractions database management
- 🤖 AI-powered features for content analysis
- 📊 Behavior analytics and user tracking
- 🗺️ Auto-geocoding for location data
- ⚡ Real-time dashboard

**Note:** This is a temporary deployment that will be accessible until this Colab session ends.

## 🛠️ Setup and Installation

In [ ]:
# Clone the repository
!git clone https://github.com/athipan1/Database_painaidee.git
%cd Database_painaidee

In [ ]:
# Install required Python packages
!pip install flask flask-sqlalchemy psycopg2-binary celery redis python-dotenv requests gunicorn
!pip install matplotlib seaborn gradio

# Install ngrok for tunneling
!pip install pyngrok

# Install PostgreSQL for local database
!apt update
!apt install -y postgresql postgresql-contrib redis-server

## 🗃️ Database Setup

In [ ]:
# Setup PostgreSQL
import os
import subprocess
import time

# Start PostgreSQL service
!service postgresql start

# Setup database user and database
!sudo -u postgres createuser --superuser root
!sudo -u postgres createdb painaidee_db

# Start Redis service
!service redis-server start

print("✅ Database services started")

In [ ]:
# Create environment configuration
env_content = """
FLASK_ENV=development
SECRET_KEY=colab-demo-secret-key-123
DATABASE_URL=postgresql://root@localhost:5432/painaidee_db
POSTGRES_DB=painaidee_db
POSTGRES_USER=root
POSTGRES_PASSWORD=
REDIS_URL=redis://localhost:6379/0
CELERY_BROKER_URL=redis://localhost:6379/0
DEBUG=True
PAGINATION_ENABLED=true
PAGINATION_PAGE_SIZE=20
API_TIMEOUT=30
"""

with open('.env', 'w') as f:
    f.write(env_content.strip())

print("✅ Environment configuration created")

## 🚀 Initialize Application

In [ ]:
# Initialize the database
!python init_db.py
print("✅ Database initialized")

## 🌐 Setup ngrok Tunnel

ngrok will create a public URL to access your Flask app running on Colab.

In [ ]:
from pyngrok import ngrok
import threading
import time

# Set up ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(f"\n🌐 ngrok tunnel created!")
print(f"📱 Access your API at: {public_url}")
print(f"📊 Dashboard available at: {public_url}/api/dashboard/")
print(f"ℹ️  API information at: {public_url}/")
print(f"\n⚠️  Note: This URL will be active until this Colab session ends.")

## ▶️ Run the Flask Application

In [ ]:
# Start Celery worker in background
import subprocess
import threading
import os

def start_celery():
    os.system('celery -A tasks.celery worker --loglevel=info --detach')

# Start Celery worker
celery_thread = threading.Thread(target=start_celery)
celery_thread.daemon = True
celery_thread.start()

print("✅ Celery worker started")
time.sleep(3)

In [ ]:
# Run the Flask application
from app import create_app
import threading

# Create Flask app
flask_app = create_app()

def run_flask():
    flask_app.run(
        host='0.0.0.0',
        port=port,
        debug=False,  # Disable debug in threading
        use_reloader=False,  # Disable reloader in Colab
        threaded=True
    )

# Start Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Wait a moment for the server to start
time.sleep(5)

print(f"\n🚀 Flask application is now running!")
print(f"\n📱 **Access your API:**")
print(f"   Main API: {public_url}")
print(f"   Dashboard: {public_url}/api/dashboard/")
print(f"   Health Check: {public_url}/api/health")
print(f"   Attractions: {public_url}/api/attractions")
print(f"\n🔗 **API Endpoints to try:**")
print(f"   GET  {public_url}/api/attractions")
print(f"   POST {public_url}/api/attractions/sync")
print(f"   GET  {public_url}/api/dashboard/stats")
print(f"\n⏰ Keep this cell running to maintain the server...")

## 🧪 Test the API

In [ ]:
import requests
import json

# Test API endpoints
base_url = str(public_url)

print("🧪 Testing API endpoints...\n")

# Test health endpoint
try:
    response = requests.get(f"{base_url}/api/health")
    print(f"✅ Health Check: {response.status_code}")
    print(f"   Response: {response.json()}\n")
except Exception as e:
    print(f"❌ Health Check failed: {e}\n")

# Test main API info
try:
    response = requests.get(f"{base_url}/")
    print(f"✅ API Info: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"   Message: {data.get('message', 'N/A')}")
        print(f"   Version: {data.get('version', 'N/A')}\n")
except Exception as e:
    print(f"❌ API Info failed: {e}\n")

# Test attractions endpoint
try:
    response = requests.get(f"{base_url}/api/attractions")
    print(f"✅ Attractions: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"   Total attractions: {len(data.get('attractions', []))}")
except Exception as e:
    print(f"❌ Attractions failed: {e}")

print(f"\n🌐 Your API is accessible at: {public_url}")
print(f"📊 Dashboard: {public_url}/api/dashboard/")

## 📊 Load Sample Data (Optional)

In [ ]:
# Trigger data sync to populate the database
try:
    response = requests.post(f"{base_url}/api/attractions/sync")
    print(f"📊 Data Sync Status: {response.status_code}")
    if response.status_code == 200:
        result = response.json()
        print(f"   Sync Result: {result}")
    else:
        print(f"   Response: {response.text}")
except Exception as e:
    print(f"❌ Data sync failed: {e}")

print(f"\n🔄 Sync completed! Check the dashboard at: {public_url}/api/dashboard/")

## 🔄 Keep Server Running

**Important:** Keep the cell below running to maintain your server. 

The API will be accessible at the ngrok URL until you stop this cell or the Colab session ends.

In [ ]:
# Keep the server running
import time

print(f"🚀 Server is running at: {public_url}")
print(f"📊 Dashboard: {public_url}/api/dashboard/")
print(f"\n⏰ Server will keep running...")
print(f"\n🛑 To stop the server, interrupt this cell (Runtime → Interrupt execution)")

try:
    while True:
        time.sleep(30)
        print(f"⚡ Server alive at {time.strftime('%H:%M:%S')} - {public_url}")
except KeyboardInterrupt:
    print("\n🛑 Server stopped by user")
    ngrok.disconnect(public_url)
    print("🔌 ngrok tunnel disconnected")

## 📖 Usage Guide

### API Endpoints Available:

**Core API:**
- `GET /` - API information
- `GET /api/health` - Health check
- `GET /api/attractions` - Get all attractions
- `POST /api/attractions/sync` - Sync data from external API

**Dashboard:**
- `GET /api/dashboard/` - Web dashboard interface
- `GET /api/dashboard/stats` - Dashboard statistics

**AI Features:**
- `POST /api/ai/keywords/extract` - Extract keywords
- `GET /api/ai/recommendations/{user_id}` - Get recommendations

### Sample Usage:

```python
import requests

# Get all attractions
response = requests.get(f"{your_ngrok_url}/api/attractions")
attractions = response.json()

# Sync new data
response = requests.post(f"{your_ngrok_url}/api/attractions/sync")
sync_result = response.json()
```

### Notes:
- 🕒 This deployment is temporary and will end when the Colab session closes
- 🌐 The ngrok URL is publicly accessible but randomly generated
- 🔄 For persistent deployment, consider using the Hugging Face Spaces option
- 📱 The dashboard provides a web interface for easier interaction

### For Full Features:
Visit the main repository: https://github.com/athipan1/Database_painaidee